In [20]:
import pandas as pd
import numpy as np
import wrangle
import warnings
warnings.filterwarnings("ignore")

import wrangle
from pydataset import data

1) Load the tips dataset.

In [29]:
tips = data('tips')

In [44]:
# split my tips data
tips_train, tips_validate, tips_test = wrangle.split_continuous(tips)

train -> (136, 7)
validate -> (59, 7)
test -> (49, 7)


1a) Create a column named tip_percentage. This should be the tip amount divided by the total bill.

In [71]:
tips_train['tip_percentage'] = tips_train['tip'] / tips_train['total_bill']

In [72]:
tips_train.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage,price_per_person
19,16.97,3.50,Female,No,Sun,Dinner,3,0.206246,0.015597
173,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345,0.006664
119,12.43,1.80,Female,No,Thur,Lunch,2,0.144811,0.011425
29,21.70,4.30,Male,No,Sat,Dinner,2,0.198157,0.019945
238,32.83,1.17,Male,Yes,Sat,Dinner,2,0.035638,0.030175


1b) Create a column named price_per_person. This should be the total bill divided by the party size.

In [73]:
tips_train['price_per_person'] = tips_train['total_bill'] / tips_train['size']

In [74]:
tips_train.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage,price_per_person
19,16.97,3.50,Female,No,Sun,Dinner,3,0.206246,5.656667
173,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345,3.625000
119,12.43,1.80,Female,No,Thur,Lunch,2,0.144811,6.215000
29,21.70,4.30,Male,No,Sat,Dinner,2,0.198157,10.850000
238,32.83,1.17,Male,Yes,Sat,Dinner,2,0.035638,16.415000


In [76]:
# Split data sets into target and features
X_train = tips_train.drop(columns=['tip'])
y_train = tips_train['tip']
X_validate = tips_validate.drop(columns=['tip'])
y_validate = tips_validate['tip']
X_test = tips_test.drop(columns=['tip'])
y_test = tips_test['tip']

1c) Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount? The tip percentage?

In [16]:
# tip percentage and price per person

1d) Use all the other numeric features to predict tip amount. Use select k best and recursive feature elimination to select the top 2 features. What are they?

In [77]:
X_train_num = X_train.drop(columns=['sex', 'smoker', 'day', 'time'])

In [78]:
X_train_num.head()

,total_bill,size,tip_percentage,price_per_person
19,16.97,3,0.206246,5.656667
173,7.25,2,0.710345,3.625000
119,12.43,2,0.144811,6.215000
29,21.70,2,0.198157,10.850000
238,32.83,2,0.035638,16.415000


In [79]:
# selecting two most predictive features using SelectKBest

from sklearn.feature_selection import SelectKBest, f_regression

In [80]:
# parameters: f_regression stats test, give me 8 features
f_selector = SelectKBest(f_regression, k=2)

# find the top 8 X's correlated with y
f_selector.fit(X_train_num, y_train)

# boolean mask of whether the column was selected or not. 
feature_mask = f_selector.get_support()

# get list of top K features. 
f_feature = X_train_num.iloc[:,feature_mask].columns.tolist()

In [81]:
f_feature

['total_bill', 'size']

In [84]:
# selecting two most predictive feature using Recursive Feature Elimination

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

# initialize the ML algorithm
lm = LinearRegression()

# create the rfe object, indicating the ML object (lm) and the number of features I want to end up with. 
rfe = RFE(lm, 2)

# fit the data using RFE
rfe.fit(X_train_num, y_train)  

# get the mask of the columns selected
feature_mask = rfe.support_

# get list of the column names. 
rfe_feature = X_train_num.iloc[:,feature_mask].columns.tolist()

In [83]:
rfe_feature

['size', 'tip_percentage']